In [4]:
from pycocotools.coco import COCO
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

In [2]:
annFile = "HotelsCocos/hotels.json"

In [ ]:
def download_img(id):
    import requests, json
    s = requests.Session()
    headers = {"Accept":"application/json", "Content-Type": "application/json"}
    auth_url = 'http://52.11.47.117:5000/api/user/login'
    auth_payload = {"username":"vision","password":"vision"}
    auth_result = s.post(auth_url, headers=headers, data=json.dumps(auth_payload))
    root = "HotelsAnns/{}.jpg".format(id)
    if auth_result.ok:
        m = "http://52.11.47.117:5000/api/image/{}?asAttachment=false&thumbnail=false".format(id)
        img_r = s.get(m)
        with open(root, 'wb') as f:
            f.write(img_r.content)
    return root

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.13s)
creating index...
index created!


In [4]:
cats = coco.loadCats(coco.getCatIds())
# for cat in cats:
#     cat['id'] = cat['id'] + 141

In [ ]:
imgs = coco.loadImgs(coco.getImgIds())

In [ ]:
test = []
for img in tqdm(imgs):
    fn = img['file_name'].replace(".jpg", ".png")
    img_arr = io.imread(download_img(img['id']))
    height, width = img_arr.shape[0], img_arr.shape[1]
    mask = np.zeros((height,width), np.uint8)

    anns = coco.loadAnns(ids=coco.getAnnIds(imgIds=img['id']))
    anns = sorted(anns, key=lambda x : x['area'])[::-1]

    for ann in anns:
        binary_mask = coco.annToMask(ann)
        where = np.argwhere(binary_mask == 1)
        for w in where:
            mask[w[0], w[1]] = ann['category_id'] + 141
    # mask[mask == 0] = 1
    io.imsave("HotelsAnns/" + fn, mask)
max(test)


In [ ]:
### CREATE COLOR FILE ###

In [ ]:
from scipy.io import loadmat
import csv


In [ ]:
c = loadmat('data/color150.mat')['colors']
colors = []
with open("data/object150_info.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for idx, row in enumerate(reader):
        colors.append((row[-1], list(c[idx])))

In [ ]:
for idx, cat in enumerate(cats):
    hex = cat['color'].lstrip('#')
    rgb = list(tuple(int(hex[i:i+2], 16) for i in (0, 2, 4)))
    colors.append((cat['name'], rgb))


In [ ]:
import pickle
with open("colors_superset.pckl", "wb") as pckl:
    pickle.dump(colors, pckl)

In [10]:
import json
test = []
with open("data/past_experiments/validation_162-class_Hotels_and_ADE.odgt", "r") as fp:
    # line = json.load(fp.readline())
    line = fp.readline()
    cnt = 1
    while line:
        line = json.loads(line)
        img = io.imread(line['fpath_segm'])
        test.append((np.amax(img), line['fpath_segm']))
        line = fp.readline()
        # cnt += 1
max(test)


(162, './imgs/HotelsAdjusted/0ROMMIE_-1_5736_traffickcam_2629252_GREY_SEG.png')

In [ ]:
### COMBINE COLORS ###

In [ ]:
ade_colors = loadmat('data/color150.mat')['colors']
ade_names= []
with open("data/object150_info.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for idx, row in enumerate(reader):
        ade_names.append(str(row[-1]))

In [ ]:
hotels_names = []
for cat in  cats:
    hex = cat['color'].lstrip('#')
    rgb = list(tuple(int(hex[i:i+2], 16) for i in (0, 2, 4)))
    hotels_names.append(cat['name'])

In [ ]:
np = set()
present = set()
gt = {(x, hotels_names.index(x) + 1) for x in hotels_names}
new = []
for b in ade_names:
    for a in hotels_names:
        matches = b.split(";")
        if any(x in a for x in matches):
            present.add((a, hotels_names.index(a) + 1))
            new.append((hotels_names.index(a) + 1, ade_names.index(b) + 1))
        else:
            np.add(a)
diff = gt - present
for i in list(diff):
    new.append((i[1],))

In [ ]:
notPresent = set()
present = set()

for b in hotels_names:
    matches = b.split(" / ")
    for a in ade_names:
        if any(x in a for x in matches):
            present.add(b)
            break
            # pass
        # else:
        #     notPresent.add(b)
present


In [ ]:
a = "test;fuck"
matches = ["fuck", "bag"]
if any(x in a for x in matches):
    print("yay")

In [ ]:
from glob import glob
import os
imgs = [f for f in glob("/pless_nfs/home/mdt_/semantic-segmentation-pytorch/HotelsData/*.png") if "GREY" in f]
test = []
for img in tqdm(imgs, desc='Adjusting Hotels'):
    s = img.split("_GREY_SEG")
    os.rename(img, s[0]+".png")
max(test)

In [8]:
import json
l = [json.loads(x.rstrip()) for x in open("data/current_experiments/validation_162-class_Hotels_and_ADE.odgt", 'r')]

In [38]:
-1* int(len(l3) * .33)

-58

In [19]:
import os
l2 = [f['fpath_segm'] for f in l]
l3 = ["./imgs/Hotels150/" + str(f) for _, __, files in os.walk("./imgs/Hotels150") for f in files]

# -class_Hotels_and_ADE.odgt", "w") as f:
# ][:58])
l3 = set(l3[:int(len(l3) * .66)])


In [20]:
import random
import json
ade = [json.loads(x.rstrip()) for x in open("data/past_experiments/training.odgt", 'r')]
n = 2000

ade_imgs = [f['fpath_segm'] for f in ade]
random.shuffle(ade_imgs)
ade = [os.path.join("data",ade_imgs[f]) for f in range(n)]

In [21]:
rem = list(l3-set(l2))
keys = ["fpath_img", "fpath_segm", "width", "height"]
with open("data/current_experiments/training_162-class_Hotels_and_ADE.odgt", "w") as f:
    total = ade + rem
    for img in ade:
        try:
            op = io.imread(img)
        except:
            print(f"bad file: {img}")
            continue
        height, width = op.shape[0], op.shape[1]
        s = f'''{"{"}"fpath_img": "{img.replace("annotations", "images").replace("png", "jpg")}", "fpath_segm": "{img}", "width": {width}, "height": {height}{"}"}\n'''
        f.write(s)
    for img in rem:
        try:
            op = io.imread(img)
        except:
            print(f"bad file: {img}")
            continue
        height, width = op.shape[0], op.shape[1]
        path = "../../datasets/Hotels-50K/images/train/" + img.replace("_GREY_SEG", "").split("_", 1)[1].replace("_", "/").replace("travel/website", "travel_website").replace("png", "jpg")
        if "train/train" in path:
            print(path)
            print(img)
            break
        s = f'''{"{"}"fpath_img": "{path}", "fpath_segm": "{img}", "width": {width}, "height": {height}{"}"}\n'''
        f.write(s)
with open("data/current_experiments/validation_162-class_Hotels_and_ADE.odgt", "w") as f:
    for img in list(l2):
        op = io.imread(img)
        height, width = op.shape[0], op.shape[1]
        path = "../../datasets/Hotels-50K/images/train/" + img.replace("_GREY_SEG", "").split("_", 1)[1].replace("_", "/").replace("travel/website", "travel_website").replace("png", "jpg")
        s = f'''{"{"}"fpath_img": "{path}", "fpath_segm": "{img}", "width": {width}, "height": {height}{"}"}\n'''
        f.write(s)





bad file: ./imgs/Hotels150/0ROMMIE_-1_5736_traffickcam_2629252.png


In [ ]:
import pickle
# col = getColors()
colors = []
names = {}
with open("Colors/colors_exp15-18.pckl", "wb") as p2:
    test = []
    for i in cats:
        hex = i['color'].lstrip('#')
        rgb = list(tuple(int(hex[i:i+2], 16) for i in (0, 2, 4)))
        test.append((rgb, i['name']))
    pickle.dump(test, p2)
    

In [ ]:
colors = []
names = {}
with open("Colors/colors_Hotels_ande.pckl", "rb") as p:
    data = pickle.load(p)
    for idx, d in enumerate(data):
        colors.append(np.array(d[0], np.uint8))
        names[idx+1] = d[1]

In [ ]:
### Montages ### 


In [ ]:
import random
import cv2
from imutils import build_montages
from imutils import paths

In [ ]:
path = "../yolov3/traffickcam"
save = "montages/yolo/"
n = 50

In [ ]:
imagePaths = list(paths.list_images(path))
random.shuffle(imagePaths)
imagePaths = imagePaths[:n]

In [ ]:
images = []
# loop over the list of image paths
for imagePath in imagePaths:
	# load the image and update the list of images
	image = cv2.imread(imagePath)
	images.append(image)
# construct the montages for the images
montages = build_montages(images, (300, 300), (5, 5))

In [ ]:
mon = list(paths.list_images(save))
for m in montages:
    cv2.imwrite(save + str(len(mon)) + ".png", m)
